In [1]:
import numpy as np
import pandas as pd
import chardet as ct

In [2]:
# needed as encodings of different files can be different and can cause problems
with open('ml-1m/movies.dat','rb') as rawdata:
    result=ct.detect(rawdata.read())
    print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [3]:
movies=pd.read_csv('ml-1m/movies.dat',sep='::',engine='python',header=None,names=['Index','Movie_name','Genre'],index_col='Index',encoding=result['encoding'])

In [4]:
movies.nunique()

Movie_name    3883
Genre          301
dtype: int64

In [5]:
movies.tail()

,Movie_name,Genre
Index,,
3948,Meet the Parents (2000),Comedy
3949,Requiem for a Dream (2000),Drama
3950,Tigerland (2000),Drama
3951,Two Family House (2000),Drama
3952,"Contender, The (2000)",Drama|Thriller


In [6]:
movies.head()

,Movie_name,Genre
Index,,
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy


In [7]:
movies['Year']=movies['Movie_name'].str.extract(r'([0-9]{4})')
# Why the extra parenthesis after: "[0-9]{4}"? as extract requires a capture grp
# This capture grp is identified inside the parenthesis only, else it doesn't understand
# The regex syntax is right in itself, its the extract method that causes issues

movies['Movie_name']=movies['Movie_name'].str.replace(r'\((\d{4})\)','',regex=True)
# Notice I used different regex but both are basically the same
# Also notice, replace doesn't need a capture grp like extract(no paranthesis to end the regex)

In [8]:
movies.sample(10)

,Movie_name,Genre,Year
Index,,,
2929,Reds,Drama,1981
336,"Walking Dead, The",Drama|War,1995
3891,Turn It Up,Crime|Drama,2000
2609,"King of Masks, The (Bian Lian)",Drama,1996
233,Exotica,Drama,1994
3468,"Hustler, The",Drama,1961
2917,Body Heat,Crime|Thriller,1981
746,Force of Evil,Film-Noir,1948
2753,"Bedroom Window, The",Thriller,1987


In [9]:
# Lets turn "Hobbit, The" type names to "The Hobbit" type
movies['Movie_name']=movies['Movie_name'].str.replace(r'^(.*), (The|An|A)',r'\2 \1',regex=True)
# Here \2 and \1 represents the 2 capture group from the earlier regex
# These are called backreferences, where \2 represents 2nd capture grp

In [10]:
movies.index.name="Movie_ID" # Renamed from index to Movie_ID
movies.sample(10)

,Movie_name,Genre,Year
Movie_ID,,,
149,Amateur,Crime|Drama|Thriller,1994
3554,Love and Basketball,Drama|Romance,2000
2677,Buena Vista Social Club,Documentary,1999
2366,King Kong,Action|Adventure|Horror,1933
115,Happiness Is in the Field,Comedy,1995
46,How to Make an American Quilt,Drama|Romance,1995
2382,Police Academy 5: Assignment: Miami Beach,Comedy,1988
3210,Fast Times at Ridgemont High,Comedy,1982
1693,Amistad,Drama,1997


In [11]:
movies_new=movies.copy()

In [12]:
# Now for seperating genres in a new dataframe
movies_new['Genre']=movies_new['Genre'].str.split('|')

# Explode needs a list of string that's why we split here

In [13]:
movies_new.sample(10)

,Movie_name,Genre,Year
Movie_ID,,,
125,Flirting With Disaster,[Comedy],1996
3068,The Verdict,[Drama],1982
1330,April Fool's Day,"[Comedy, Horror]",1986
2922,Hang 'em High,[Western],1967
2989,For Your Eyes Only,[Action],1981
2859,Stop Making Sense,[Documentary],1984
980,In the Line of Duty 2,[Action],1987
449,Fear of a Black Hat,[Comedy],1993
599,The Wild Bunch,[Western],1969


In [14]:
exploded=movies_new.explode('Genre')
"""
Use of explode:
It takes a single row with a list-like value (e.g., a list of genres) and 
creates a new row for each item in that list. 
The data in the other columns is duplicated.
"""

'\nUse of explode:\nIt takes a single row with a list-like value (e.g., a list of genres) and \ncreates a new row for each item in that list. \nThe data in the other columns is duplicated.\n'

In [15]:
exploded.head(10)

,Movie_name,Genre,Year
Movie_ID,,,
1,Toy Story,Animation,1995
1,Toy Story,Children's,1995
1,Toy Story,Comedy,1995
2,Jumanji,Adventure,1995
2,Jumanji,Children's,1995
2,Jumanji,Fantasy,1995
3,Grumpier Old Men,Comedy,1995
3,Grumpier Old Men,Romance,1995
4,Waiting to Exhale,Comedy,1995


In [16]:
with open('ml-1m/ratings.dat','rb') as rat:
    result=ct.detect(rat.read())
    # print(result)

head=['UserID','Movie_id','Ratings','Timestamp']
ratings=pd.read_csv('ml-1m/ratings.dat',sep='::',names=head,index_col='Movie_id',encoding=result['encoding'])

/tmp/ipykernel_18653/477480043.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings=pd.read_csv('ml-1m/ratings.dat',sep='::',names=head,index_col='Movie_id',encoding=result['encoding'])


In [17]:
ratings.head()

,UserID,Ratings,Timestamp
Movie_id,,,
1193,1,5,978300760
661,1,3,978302109
914,1,3,978301968
3408,1,4,978300275
2355,1,5,978824291


In [18]:
# Currently the timestamp is in Unix timestamp also known as the epoch timestamp
ratings['Timestamp']=pd.to_datetime(ratings['Timestamp'],unit='s')
# 's' specifies seconds

In [19]:
ratings.index.name="Movie_ID"
ratings.head()
# Notice now timestamp has both date and time, lets seperate them out

,UserID,Ratings,Timestamp
Movie_ID,,,
1193,1,5,2000-12-31 22:12:40
661,1,3,2000-12-31 22:35:09
914,1,3,2000-12-31 22:32:48
3408,1,4,2000-12-31 22:04:35
2355,1,5,2001-01-06 23:38:11


In [20]:
# ratings['Date']=ratings['Timestamp'].str.extract(r'(\d{4}-\d{2}-\d{2})')
"""Can't do the above as a datetime object can't be converted to string type"""

ratings['Date']=ratings['Timestamp'].dt.date
ratings['Timestamp']=ratings['Timestamp'].dt.time
ratings.rename(columns={'Timestamp':'Time'},inplace=True)
ratings.head()

,UserID,Ratings,Time,Date
Movie_ID,,,,
1193,1,5,22:12:40,2000-12-31
661,1,3,22:35:09,2000-12-31
914,1,3,22:32:48,2000-12-31
3408,1,4,22:04:35,2000-12-31
2355,1,5,23:38:11,2001-01-06


In [21]:
movies.index.max()

np.int64(3952)

In [50]:
movies.isna().sum()

Movie_name    0
Genre         0
Year          0
dtype: int64

In [22]:
ratings.index.max()

3952

#### both have the same number of entries BUT the index datatype is different

In [40]:
ratings['UserID'].unique()
ratings=ratings.reset_index() # Great method, resets the indexes to default 0-total entries, while the old values in index get pushed to a new column with same name
"""
Why did i do it?
well because i made an incorrect assumptions that the Movie_ID are unique, without even checking the total entries of each columns including Movie_ID
"""
ratings.head()

,index,Movie_ID,UserID,Ratings,Time,Date
0,0,1193,1,5,22:12:40,2000-12-31
1,1,661,1,3,22:35:09,2000-12-31
2,2,914,1,3,22:32:48,2000-12-31
3,3,3408,1,4,22:04:35,2000-12-31
4,4,2355,1,5,23:38:11,2001-01-06


In [48]:
ratings.isna().sum()

index       0
Movie_ID    0
UserID      0
Ratings     0
Time        0
Date        0
dtype: int64

#### Cleaning users:

In [41]:
with open('ml-1m/users.dat','rb') as user:
    result=ct.detect(user.read())
    # print(result)
# UserID::Gender::Age::Occupation::Zip-code
head=['UserID','Gender','Age','Occupation','Zip-code']
users=pd.read_csv('ml-1m/users.dat',sep='::',names=head,encoding=result['encoding'])

/tmp/ipykernel_18653/877249055.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users=pd.read_csv('ml-1m/users.dat',sep='::',names=head,encoding=result['encoding'])


In [42]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [47]:
users.isna().sum()

UserID        0
Gender        0
Age           0
Occupation    0
Zip-code      0
dtype: int64

In [46]:
users['Gender'].value_counts()

Gender
M    4331
F    1709
Name: count, dtype: int64

Users seem to be pretty clean already, so lets focus on merge tables and pivot tables next!